<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Scale Decision Processes with Plugins  

Decision Intelligence applied in this module:  
* Adding current knowledge (internet) to gather intelligence for more informed decisions  
* Decision Scenario: Investigate a specific decision made after the recent NBA sports season.  

Semantic Kernel Plugins are complete building blocks of functionality for AI orchestration. Plugins consist of multiple single responsibility functions (semantic or native). A plugin brings together all of the tools it needs a GenAI model, agent or assistant to use under a single construct. Below is an image of a Writer Plugin that consists of multiple functions: Brainstorming, E-mail generation, Translation etc. 

<img style="display: block; margin: auto;" src="https://learn.microsoft.com/en-us/semantic-kernel/media/writer-plugin-example.png">


Plugins are very powerful because they allow an AI architect to compose functionality from many smaller functions. Plugins can be crafted in several different ways:
* Use multiple native functions (C# methods) 
* Use multiple semantic functions exposed in prompt files, YAML configurations etc.  
* Hybrid combinations of multiple native or semantic functions  
* Remote API services called via functions (database, internet knowledge graph, APIs)  
* Remote platform services (calling Azure Logic Apps, which includes over 1,400 connectors)  

Learn more about Semantic Kernel Plugins: https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/ 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.38"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.38-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.38-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.38.0 Microsoft.SemanticKernel.Plugins.Core, 1.38.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.38.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Limitations of Current Knowledge of LLMs 

Decisions are being made in our environment and by own selves non-stop. In order to investigate and analyze current decisions, GenAI LLMs need current access to the knowledge graphs that persist this information. **In decision-making, this phase is called "Gathering Relevant Intelligence"** 

Execute the cell below to attempt to investigate what coaching decision did the Phoenix Suns make after the 2024 NBA playoffs?"

In [3]:
// Prompt Template for Decision Iquiry
var promptTemplateString = """
What decision did {{$companyName}} make in September 2024?
Only answer if you know the answer. Do not halliucinate.
""";

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
};

var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["companyName"] = "Intel"
};

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

I'm sorry, but I cannot provide information about events or decisions made in September 2024, as my knowledge only extends up to October 2023.

### Step 3 - Acessing Current Knowledge with a Bing Web Search Plugin

In order to inspect current decisions being made, an LLM needs access to a knowledge graph (internet) that includes recent information and news. Conveniently Semantic Kernel includes a web plugin that exposes access to the Bing Search knowledge graph. This allows Semantic Kernel to gather relevant intelligence for the decision topic using Bing's internet indexes.  

Execute the two cells below to create and import the Bing connector to Semantic Kernel instance. **Notice that just by adding the Bing plugin, the link between Semantic Kernel knowledge and the LLM is still not made.**    

In [4]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

#pragma warning disable SKEXP0050
var bingConnector = new BingConnector(bingSearchAPIKey);
var bingSearchPlugin = new WebSearchEnginePlugin(bingConnector);

semanticKernel.ImportPluginFromObject(bingSearchPlugin, "bing");
#pragma warning restore SKEXP0050

In [5]:
// Still doesn't work, even though Bing Knowledge Graph is enabled
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

I'm sorry, but I cannot provide information about events or decisions made in September 2024, as my knowledge only extends up to October 2023.

### Step 4 - Proving Semantic Kernel the ability to call the Bing Search Plugin

Notice after importing the Bing Search plugin into the Semantic Kernel instance, the LLM still was not able to answer the question. What is missing? **In addition to importing the appropriate plugin, the LLM needs to be "allowed" to automatically invoke imported Semantic Kernel functions. This is done by overriding the ToolCallBehavior property to AutoInvokeFunctions.**  

Execute the cell below to allow auto invocation of Semantic Kernel functions. The configuration now allows Semantic Kernel to find the right tool (Bing Search) and automatically invoke it. Compare this output with the previous output(s), notice the GenAI LLM was able to answer the question.  

In [6]:
// Prompt Template for Decision Iquiry
var promptTemplateString = """
What decision did {{$companyName}} make in September 2024?
Only answer if you know the answer. Do not halliucinate.
""";

// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // NEW: Enable Auto Invoking of Kernel Functions
    // Allows it to call the available Bing Plugin
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["companyName"] = "Intel"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

In September 2024, Intel decided to postpone its annual Innovation event, which was originally scheduled for September 24-25, until 2025.

### Step 5 - Understanding Auto Function Calling Behavior

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [7]:
// Method for discovering the available functions loaded into Semantic Kernel via metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: bing
  Function Name: Search
  Function Description: Perform a web search.
    Function Parameters:
      - query: Search query
        default: ''
      - count: Number of results
        default: '10'
      - offset: Number of results to skip
        default: '0'

Plugin: bing
  Function Name: GetSearchResults
  Function Description: Perform a web search and return complete results.
    Function Parameters:
      - query: Text to search for
        default: ''
      - count: Number of results
        default: '1'
      - offset: Number of results to skip
        default: '0'

